In [1]:
import stockutils, basicutils, config
import pandas_datareader.data as web, re, sqlite3, datetime

In [2]:
class StockData:
    def __init__(self, update = False):
        self.company_table = stockutils.company_table(update = update)
        
    @basicutils.memo
    def lookup_ticker(self, name, se = None):
        """Converts company_name to ticker symbol. Returns a tuple of (symbol, company name) 
        or returns an empty table if company or ticker is not found."""
        conn = sqlite3.connect(config.sql_path)
        c = conn.cursor()
        result_table = []
        result_table.extend(stockutils.sql_search(name, ['symbol','name','se'], c, se=se))
        if not result_table:
            print('Switching to regular expressions for ' + name + ' lookup.')
            found = self.fuzzy_search(name)
            if found:
                for item in found:
                    result_table.extend(stockutils.sql_search(item, ['symbol','name','se'], c, se=se))
            else:
                result_table.append(('N/A', name))
            conn.close()
        return result_table
    
    #soon to be moved to matchutils.py
    def fuzzy_search(self,name):
        found_names = []
        for name in [name, name.upper()]:
            query = r".*" + r".*".join(list(name)) + r".*"
            for row in self.company_table.keys():
                match = re.search(query,row)
                if match:
                     found_names.append(match.group())
        return found_names
    
    @basicutils.memo   
    def ipo_year(self,company_name):
        list_years = []
        conn = sqlite3.connect(config.sql_path)
        c = conn.cursor()
        return stockutils.sql_search(company_name, ['ipoyear', 'symbol'], c) 
    
            
#       def stock_price(self, ticker, date):
            
        
        

In [3]:
basicutils.import_cache('data/cached_searches.json')

In [4]:
w = StockData()

In [5]:
w.lookup_ticker('Apple')

Switching to regular expressions for Apple lookup.


[('AMAT', 'Applied Materials, Inc.', 'nasdaq'),
 ('AMCC', 'Applied Micro Circuits Corporation', 'nasdaq'),
 ('GCP', 'GCP Applied Technologies Inc.', 'nyse'),
 ('APLE', 'Apple Hospitality REIT, Inc.', 'nyse'),
 ('AGTC', 'Applied Genetic Technologies Corporation', 'nasdaq'),
 ('AIT', 'Applied Industrial Technologies, Inc.', 'nyse'),
 ('AAPL', 'Apple Inc.', 'nasdaq'),
 ('ARCI', 'Appliance Recycling Centers of America, Inc.', 'nasdaq'),
 ('AAOI', 'Applied Optoelectronics, Inc.', 'nasdaq'),
 ('APDN', 'Applied DNA Sciences Inc', 'nasdaq'),
 ('SAIC', 'SCIENCE APPLICATIONS INTERNATIONAL CORPORATION', 'nyse')]

In [6]:
w.lookup_ticker('Nvidia', 'nasdaq')

Switching to regular expressions for Nvidia lookup.


[('NVDA', 'NVIDIA Corporation', 'nasdaq')]

In [7]:
basicutils.bulk_search(w.lookup_ticker,['Intel Corporation', 'Nvidia', 'Apple', 'Microsoft', 'Becton Dickinson'], se = 'nasdaq')

Switching to regular expressions for Nvidia lookup.
Switching to regular expressions for Apple lookup.
Switching to regular expressions for Microsoft lookup.
Switching to regular expressions for Becton Dickinson lookup.


[('INTC', 'Intel Corporation', 'nasdaq'),
 ('NVDA', 'NVIDIA Corporation', 'nasdaq'),
 ('AMAT', 'Applied Materials, Inc.', 'nasdaq'),
 ('AMCC', 'Applied Micro Circuits Corporation', 'nasdaq'),
 ('AGTC', 'Applied Genetic Technologies Corporation', 'nasdaq'),
 ('AAPL', 'Apple Inc.', 'nasdaq'),
 ('ARCI', 'Appliance Recycling Centers of America, Inc.', 'nasdaq'),
 ('AAOI', 'Applied Optoelectronics, Inc.', 'nasdaq'),
 ('APDN', 'Applied DNA Sciences Inc', 'nasdaq'),
 ('MSFT', 'Microsoft Corporation', 'nasdaq')]

In [8]:
w.lookup_ticker("Dick's")

Switching to regular expressions for Dick's lookup.


[('DKS', "Dick's Sporting Goods Inc", 'nyse')]

In [11]:
w.lookup_ticker('Becton', se = 'nyse')

Switching to regular expressions for Becton lookup.


[('BNED', 'Barnes & Noble Education, Inc', 'nyse'),
 ('BIT', 'BlackRock Multi-Sector Income Trust', 'nyse'),
 ('BDX', 'Becton, Dickinson and Company', 'nyse'),
 ('WAB', 'Westinghouse Air Brake Technologies Corporation', 'nyse'),
 ('IBM', 'International Business Machines Corporation', 'nyse'),
 ('BAC', 'Bank of America Corporation', 'nyse'),
 ('BPI', 'Bridgepoint Education, Inc.', 'nyse'),
 ('JBT', 'John Bean Technologies Corporation', 'nyse'),
 ('BHE', 'Benchmark Electronics, Inc.', 'nyse'),
 ('BTZ', 'BlackRock Credit Allocation Income Trust', 'nyse'),
 ('INF', 'Brookfield Global Listed Infrastructure Income Fund', 'nyse'),
 ('BR', 'Broadridge Financial Solutions, Inc.', 'nyse'),
 ('BGE^B', 'Baltimore Gas & Electric Company', 'nyse'),
 ('BSX', 'Boston Scientific Corporation', 'nyse')]

In [13]:
basicutils.bulk_search(w.lookup_ticker,['Intel', 'BD'])

[('CSGS', 'CSG Systems International, Inc.', 'nasdaq'),
 ('BAX', 'Baxter International Inc.', 'nyse'),
 ('INXN', 'InterXion Holding N.V.', 'nyse'),
 ('PCOM', 'Points International, Ltd.', 'nasdaq'),
 ('RFDI',
  'First Trust RiverFront Dynamic Developed International ETF',
  'nasdaq'),
 ('CXRX', 'Concordia International Corp.', 'nasdaq'),
 ('LII', 'Lennox International, Inc.', 'nyse'),
 ('NXC', 'Nuveen Insured California Select Tax-Free Income Portfolio', 'nyse'),
 ('HASI', 'Hannon Armstrong Sustainable Infrastructure Capital, Inc.', 'nyse'),
 ('OII', 'Oceaneering International, Inc.', 'nyse'),
 ('JCI', 'Johnson Controls International plc', 'nyse'),
 ('PNQI', 'PowerShares Nasdaq Internet Portfolio', 'nasdaq'),
 ('CLUB', 'Town Sports International Holdings, Inc.', 'nasdaq'),
 ('LOGI', 'Logitech International S.A.', 'nasdaq'),
 ('VTN', 'Invesco Trust  for Investment Grade New York Municipal', 'nyse'),
 ('SEAC', 'SeaChange International, Inc.', 'nasdaq'),
 ('NBH', 'Neuberger Berman Interme

In [14]:
w.ipo_year('Intel Corporation')

[('n/a', 'INTC')]

In [15]:
basicutils.bulk_search(w.ipo_year,['Blackrock Core Bond Trust', 'BlackRock Credit Allocation Income Trust'])

[('2001', 'BHK'), ('2006', 'BTZ')]

In [16]:
w.lookup_ticker('Becton, Dickinson and Company')

[('BDX', 'Becton, Dickinson and Company', 'nyse')]

In [17]:
basicutils.json_dump('data/cached_searches.json')

In [18]:
basicutils.clear_cache('data/cached_searches.json')